In [ ]:
def get_example_info(line):
    # get label and 2nd argument
    ex_label_match = re.search(r'\\begin\{Example\}\{(.*)?\}\{(.*)?\}', line)
    label = ex_label_match.group(1)
    arg_2 = ex_label_match.group(2)
    return { 'label' : label, 'arg_2' : arg_2 }

In [ ]:
def convert_line(line):
    # inline math
    line = re.sub(r'\$(.*?)\$', "<m>\g<1></m>", line)
    # boldface text
    line = re.sub(r'\\textbf\{(.*?)\}', "<term>\g<1></term>", line)
    # double left single quotes
    line = re.sub(r'``', '"', line)
    # double right single quotes
    line = re.sub(r"''",'"', line)
    # single line or inline \verb
    line = re.sub(r'\\verb\|(.*?)\|', '<c>\g<1></c>', line)
    line = re.sub(r'>>', '', line)
    return line

In [ ]:
def convert_multi_verb_line(line):
    line = re.sub(r'\\\\', "\n", line) # line break character
    line = re.sub(r'\\ps', "\t", line) # tab character
    line = re.sub(r'>>', '', line) # commandline prompt character
    line = re.sub(r'\\verb\|(.*?)\|', "\t\g<1>", line)
    return line

In [ ]:
def parse_example():
    return 0
def parse_multi_line_verb():
    return 0
def parse_comments():
    return 0

In [ ]:
in_example = False
in_section = False
in_paragraph = False
in_verb = False
tab_depth = 0
last_line_type = ''
pretext_lines = []
for l in range(len(latex_lines)):
    line = latex_lines[l]
    if l < len(latex_lines)-1:
        next_line = latex_lines[l+1]
        next_line_type = get_line_type(next_line)
    else:
        next_line = ''
        next_line_type = ''
    line_type = get_line_type(line)
    if line_type not in ['comment','index']:
        # check for section
        if line_type == 'section':
            if in_section == False: # first section found
                section_info = get_section_info(line)
                in_section = True
                tab_depth += 1
            else: # new section found
                pretext_lines.append('</section>\n\n') # end last section
            pretext_lines.append('<section xml:id="' + section_info['label'] + '" xmlns:xi="http://www.w3.org/2001/XInclude">\n')
            pretext_lines.append('<title>' + section_info['title'] + '</title>')
            pretext_lines.append('\n')
        # check for new \verb
        elif last_line_type == 'blank' and line_type == 'verb':
            if next_line_type == 'blank': # single line verb
                pretext_lines.append(convert_line(line))
            else: # multi-line verb
                in_verb = True
                pretext_lines.append('<c>\n')
                pretext_lines.append(convert_multi_verb_line(line[:-1]))
        # continue \verb
        elif in_verb and line_type not in ['blank','e_example','section']:
            pretext_lines.append(convert_multi_verb_line(line[:-1]))
        # check for end \verb
        elif in_verb and line_type in ['blank','e_example','section']:
            pretext_lines.append('\n</c>\n\n')
            in_verb = False
            if line_type == 'e_example':
                in_example = False
                pretext_lines.append('</example>\n')
            elif line_type == 'section':
                pretext_lines.append('</section>\n\n') # end last section
        # check for new example
        elif line_type == 'b_example':
            in_example = True
            example_info = get_example_info(line)
            pretext_lines.append('<example xml:id="' + example_info['label'] + '">\n')
        # check for end of example
        elif line_type == 'e_example':
            in_example = False
            pretext_lines.append('\n</example>\n')
        # check for new paragraph
        elif last_line_type == 'blank' and line_type == 'other':
            in_paragraph = True
            tab_depth = tab_depth + '\t'
            pretext_lines.append('<p>\n')
            pretext_lines.append('\t ' + convert_line(line[:-1]))
        # check for end of paragraph
        elif in_paragraph and line_type == 'blank':
            pretext_lines.append('\n</p>\n\n')
            in_paragraph = False
        # check for in example or paragraph
        elif in_example or in_paragraph:
            pretext_lines.append(convert_line(line[:-1]))
        else:
            pretext_lines.append(convert_line(line))
    last_line_type = line_type

In [ ]:
for l in range(len(pretext_lines)):
    line = pretext_lines[l]
    print(line)
latex_file.close()

In [ ]:
tab_depth = 0
last_line_type = ''
pretext_lines = []
for l in range(len(latex_lines)):
    line = latex_lines[l]
    if l < len(latex_lines)-1:
        next_line = latex_lines[l+1]
        next_line_type = get_line_type(next_line)
    else:
        next_line = ''
        next_line_type = ''
    line_type = get_line_type(line)
    if line_type not in ['comment','index']:
        # check for section
        if line_type == 'section':
            if in_section == False: # first section found
                section_info = get_section_info(line)
                in_section = True
                tab_depth += 1
            else: # new section found
                pretext_lines.append('</section>\n\n') # end last section
            pretext_lines.append('<section xml:id="' + section_info['label'] + '" xmlns:xi="http://www.w3.org/2001/XInclude">\n')
            pretext_lines.append('<title>' + section_info['title'] + '</title>')
            pretext_lines.append('\n')
        # check for new \verb
        elif last_line_type == 'blank' and line_type == 'verb':
            if next_line_type == 'blank': # single line verb
                pretext_lines.append(convert_line(line))
            else: # multi-line verb
                in_verb = True
                pretext_lines.append('<c>\n')
                pretext_lines.append(convert_multi_verb_line(line[:-1]))
        # continue \verb
        elif in_verb and line_type not in ['blank','e_example','section']:
            pretext_lines.append(convert_multi_verb_line(line[:-1]))
        # check for end \verb
        elif in_verb and line_type in ['blank','e_example','section']:
            pretext_lines.append('\n</c>\n\n')
            in_verb = False
            if line_type == 'e_example':
                in_example = False
                pretext_lines.append('</example>\n')
            elif line_type == 'section':
                pretext_lines.append('</section>\n\n') # end last section
        # check for new example
        elif line_type == 'b_example':
            in_example = True
            example_info = get_example_info(line)
            pretext_lines.append('<example xml:id="' + example_info['label'] + '">\n')
        # check for end of example
        elif line_type == 'e_example':
            in_example = False
            pretext_lines.append('\n</example>\n')
        # check for new paragraph
        elif last_line_type == 'blank' and line_type == 'other':
            in_paragraph = True
            tab_depth = tab_depth + '\t'
            pretext_lines.append('<p>\n')
            pretext_lines.append('\t ' + convert_line(line[:-1]))
        # check for end of paragraph
        elif in_paragraph and line_type == 'blank':
            pretext_lines.append('\n</p>\n\n')
            in_paragraph = False
        # check for in example or paragraph
        elif in_example or in_paragraph:
            pretext_lines.append(convert_line(line[:-1]))
        else:
            pretext_lines.append(convert_line(line))
    last_line_type = line_type

In [ ]:
pretext_lines = copy_replace(latex_lines)

In [ ]:
for l in range(len(pretext_lines)):
    line = pretext_lines[l]
    print(line)
latex_file.close()